d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Hyperparameter Tuning with Random Forests

In this lab, you will convert the Airbnb problem to a classification dataset, build a random forest classifier, and tune some hyperparameters of the random forest.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you:<br>
 - Perform grid search on a random forest
 - Get the feature importances across the forest
 - Save the model
 - Identify differences between scikit-learn's Random Forest and SparkML's
 
You can read more about the distributed implementation of Random Forests in the Spark [source code](https://github.com/apache/spark/blob/master/mllib/src/main/scala/org/apache/spark/ml/tree/impl/RandomForest.scala#L42).

In [0]:
%run "../Includes/Classroom-Setup"

Initialized classroom variables & functions...

Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

Declared various utility methods: 
 Declared untilStreamIsReady( name:String ) to control workflow 
 
 All done!

## From Regression to Classification

In this case, we'll turn the Airbnb housing dataset into a classification problem to **classify between high and low price listings.**  Our `class` column will be:<br><br>

- `0` for a low cost listing of under $150
- `1` for a high cost listing of $150 or more

In [0]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

filePath = "dbfs:/mnt/training/airbnb/sf-listings/sf-listings-2019-03-06-clean.delta/"

airbnbDF = (spark.read.format("delta").load(filePath)
  .withColumn("priceClass", (col("price") >= 150).cast("int"))
  .drop("price")
)

(trainDF, testDF) = airbnbDF.randomSplit([.8, .2], seed=42)

categoricalCols = [field for (field, dataType) in trainDF.dtypes if dataType == "string"]
indexOutputCols = [x + "Index" for x in categoricalCols]

stringIndexer = StringIndexer(inputCols=categoricalCols, outputCols=indexOutputCols, handleInvalid="skip")

numericCols = [field for (field, dataType) in trainDF.dtypes if ((dataType == "double") & (field != "priceClass"))]
assemblerInputs = indexOutputCols + numericCols
vecAssembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

## Why can't we OHE?

**Question:** What would go wrong if we One Hot Encoded our variables before passing them into the random forest?

**HINT:** Think about what would happen to the "randomness" of feature selection.

## Random Forest

Create a Random Forest classifer called `rf` with the `labelCol`=`priceClass`, `maxBins`=`40`, and `seed`=`42` (for reproducibility).

It's under `pyspark.ml.classification.RandomForestClassifier` in Python and `org.apache.spark.ml.classification.RandomForestClassifier` in Scala.

In [0]:
# TODO
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="priceClass", maxBins=40, seed=42)

## Grid Search

There are a lot of hyperparameters we could tune, and it would take a long time to manually configure.

Let's use Spark's `ParamGridBuilder` to find the optimal hyperparameters in a more systematic approach [Python](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.ParamGridBuilder)/[Scala](https://spark.apache.org/docs/latest/api/scala/#org.apache.spark.ml.tuning.ParamGridBuilder).

Let's define a grid of hyperparameters to test:
  - maxDepth: max depth of the decision tree (Use the values `2, 5, 10`)
  - numTrees: number of decision trees (Use the values `10, 20, 100`)

`addGrid()` accepts the name of the parameter (e.g. `rf.maxDepth`), and a list of the possible values (e.g. `[2, 5, 10]`).

In [0]:
# TODO
from pyspark.ml.tuning import ParamGridBuilder

paramGrid = (ParamGridBuilder()
            .addGrid(rf.maxDepth, [2, 5, 10])
            .addGrid(rf.numTrees, [10, 20, 100])
            .build())

-sandbox
## Evaluator

In the past, we used a `RegressionEvaluator`.  For classification, we can use a [BinaryClassificationEvaluator](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.BinaryClassificationEvaluator) if we have two classes or [MulticlassClassificationEvaluator](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.MulticlassClassificationEvaluator) for more than two classes.

Create a `BinaryClassificationEvaluator` with `areaUnderROC` as the metric.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> [Read more on ROC curves here.](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)  In essence, it compares true positive and false positives.

In [0]:
# TODO
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol="priceClass")

## Cross Validation

We are going to do 3-Fold cross-validation, with `parallelism`=4, and set the `seed`=42 on the cross-validator for reproducibility.

Put the Random Forest in the CV to speed up the cross validation (as opposed to the pipeline in the CV) [Python](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.CrossValidator)/[Scala](https://spark.apache.org/docs/latest/api/scala/#org.apache.spark.ml.tuning.CrossValidator).

In [0]:
# TODO

from pyspark.ml.tuning import CrossValidator

cv = CrossValidator(estimator=rf, 
                    evaluator=evaluator, 
                    estimatorParamMaps=paramGrid,
                    numFolds=3, 
                    parallelism=4, 
                    seed=42)

## Pipeline

Let's fit the pipeline with our cross validator to our training data (this may take a few minutes).

In [0]:
stages = [stringIndexer, vecAssembler, cv]

pipeline = Pipeline(stages=stages)

pipelineModel = pipeline.fit(trainDF)

/databricks/spark/python/pyspark/ml/util.py:838: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

## Hyperparameter

Which hyperparameter combination performed the best?

In [0]:
cvModel = pipelineModel.stages[-1]
rfModel = cvModel.bestModel

list(zip(cvModel.getEstimatorParamMaps(), cvModel.avgMetrics))

# print(rfModel.explainParams())

Out[13]: [({Param(parent='RandomForestClassifier_4ee87563ad4c', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 2,
 Param(parent='RandomForestClassifier_4ee87563ad4c', name='numTrees', doc='Number of trees to train (>= 1).'): 10},
 0.8494609892340328),
 ({Param(parent='RandomForestClassifier_4ee87563ad4c', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 2,
 Param(parent='RandomForestClassifier_4ee87563ad4c', name='numTrees', doc='Number of trees to train (>= 1).'): 20},
 0.8450403538026396),
 ({Param(parent='RandomForestClassifier_4ee87563ad4c', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 2,
 Param(parent='RandomForestClassifier_4ee87563ad4c', name='numTrees', doc='Number of trees to train (>= 1).'): 100},
 0.8572368210894231),
 ({Param(parent='RandomForestClassifier_4ee87563ad4c', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5,
 Param(parent='RandomForestClassifier_4ee87563ad4c', name='numTrees', doc='Number of trees to train (>= 1).'): 10},
 0.8794946343548473),
 ({Param(parent='RandomForestClassifier_4ee87563ad4c', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5,
 Param(parent='RandomForestClassifier_4ee87563ad4c', name='numTrees', doc='Number of trees to train (>= 1).'): 20},
 0.8872232007414571),
 ({Param(parent='RandomForestClassifier_4ee87563ad4c', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5,
 Param(parent='RandomForestClassifier_4ee87563ad4c', name='numTrees', doc='Number of trees to train (>= 1).'): 100},
 0.8882009830669131),
 ({Param(parent='RandomForestClassifier_4ee87563ad4c', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 10,
 Param(parent='RandomForestClassifier_4ee87563ad4c', name='numTrees', doc='Number of trees to train (>= 1).'): 10},
 0.9048881178918164),
 ({Param(parent='RandomForestClassifier_4ee87563ad4c', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 10,
 Param(parent='RandomForestClassifier_4ee87563ad4c', name='numTrees', doc='Number of trees to train (>= 1).'): 20},
 0.9142624641849966),
 ({Param(parent='RandomForestClassifier_4ee87563ad4c', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 10,
 Param(parent='RandomForestClassifier_4ee87563ad4c', name='numTrees', doc='Number of trees to train (>= 1).'): 100},
 0.9180625529465041)]

## Feature Importance

In [0]:
import pandas as pd

pandasDF = pd.DataFrame(list(zip(vecAssembler.getInputCols(), rfModel.featureImportances)), columns=["feature", "importance"])
topFeatures = pandasDF.sort_values(["importance"], ascending=False)
topFeatures

Out[14]:

,feature,importance
12,bedrooms,0.156562
5,room_typeIndex,0.145999
10,accommodates,0.145594
3,neighbourhood_cleansedIndex,0.094439
13,beds,0.074897
7,host_total_listings_count,0.056620
8,latitude,0.051551
9,longitude,0.039338
16,review_scores_rating,0.033169
15,number_of_reviews,0.031997


Do those features make sense? Would you use those features when picking an Airbnb rental?

## Apply Model to test set

In [0]:
# TODO

predDF = pipelineModel.transform(testDF)
areaUnderROC = evaluator.evaluate(predDF)
print(f"Area under ROC is {areaUnderROC:.2f}")

Area under ROC is 0.92

## Save Model

Save the model to `<userhome>/rf_pipeline_model`.

In [0]:
# TODO

pipelineModel.write().overwrite().save(userhome + "/rf_pipeline_model")

## Sklearn vs SparkML

[Sklearn RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) vs `SparkML RandomForestRegressor` [Python](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.regression.RandomForestRegressor)/[Scala](https://spark.apache.org/docs/latest/api/scala/#org.apache.spark.ml.regression.RandomForestRegressor).

Look at these params in particular:
* **n_estimators** (sklearn) vs **numTrees** (SparkML)
* **max_depth** (sklearn) vs **maxDepth** (SparkML)
* **max_features** (sklearn) vs **featureSubsetStrategy** (SparkML)
* **maxBins** (SparkML only)

What do you notice that is different?

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>